# Prepare data for input

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import edward as ed

In [2]:
ed.set_seed(12227)

In [3]:
dat = pd.read_csv('data/prepared_data.csv')
print(dat.shape)
dat.head()

(687276, 7)


,stim,unit,isfirst,isrewarded,count,trial,time
0,0,1068,0,0,4,57277,0
1,0,1069,0,0,1,57277,0
2,0,1070,0,0,6,57277,0
3,0,1071,0,0,0,57277,0
4,0,1072,1,0,3,60250,0


Columns are:
- stim: original stimulus number
- unit: recorded neuron number
- isfirst, isrewarded: potential regressors of interest
- count: spike count during stimulus display period
- trial: original trial number
- time: unique stimulus code in trimmed dataset (very rare stims removed)

We need to do a couple of things to prep the data:
- turn unit codes into a 1-based unit index
- turn time into a 1-based stim index

In [4]:
# subset data for testing
#dat = dat.query("unit < 1050 & time < 300")
dat.shape

(687276, 7)

In [5]:
_, unit = np.unique(dat.unit, return_inverse=True)
_, stim = np.unique(dat.time, return_inverse=True)

print(np.min(unit), np.max(unit), len(np.unique(unit)))

print(np.min(stim), np.max(stim), len(np.unique(stim)))

0 525 526
0 797 798


In [6]:
count = dat['count'].values
Xdat = dat[['isfirst', 'isrewarded']].values
Xdat.shape

(687276, 2)

# Define the model

In [7]:
# define some needed constants
N = dat.shape[0]  # number of trials
NB = 10  # number of trials in minibatch
NU = len(np.unique(unit))  # number of units
NS = len(np.unique(stim))  # number of stims
P = Xdat.shape[1]  # number of specified regressors
K = 5  # number of latents

## Generative (p) model

In [8]:
with tf.variable_scope("pmodel"):
    A = ed.models.Normal(mu=tf.zeros(NU), sigma=tf.ones(NU), name='A')
    B = ed.models.Normal(mu=tf.zeros((NU, P)), sigma=tf.ones((NU, P)), name='B')
    C = ed.models.Normal(mu=tf.zeros((NU, K)), sigma=tf.ones((NU, K)), name='C')

    X = tf.constant(Xdat.astype('float32'))
    U = tf.constant(unit)
    S = tf.constant(stim)

    delta = ed.models.Beta(a=3 * tf.ones(K), b=tf.ones(K), name='delta')
    tf.scalar_summary('mean_delta', tf.reduce_mean(delta))
    log_delta = tf.log(delta)
    tf.scalar_summary('min_log_delta', tf.reduce_min(log_delta))
    tf.scalar_summary('mean_log_delta', tf.reduce_mean(log_delta))

    pi = tf.exp(tf.cumsum(log_delta), name='pi')
    tf.scalar_summary('min_pi', tf.reduce_min(pi))

    Z = ed.models.Bernoulli(p=tf.tile(tf.expand_dims(pi, 0), [NS, 1]), name='Z')
    tf.scalar_summary('mean_Z', tf.reduce_mean(tf.to_float(Z)))

    sig = ed.models.Normal(mu=[-0.1], sigma=[0.1], name='sig')

    lam = ed.models.Normal(mu=(tf.gather(A, U) + tf.reduce_sum(tf.gather(B, U) * X, 1) + 
           tf.reduce_sum(tf.gather(C, U) * tf.gather(tf.to_float(Z), S), 1)), 
                           sigma=tf.exp(sig), name='lam')
    tf.scalar_summary('mean_lam', tf.reduce_mean(lam))


    cnt = ed.models.Poisson(lam=tf.nn.softplus(lam), value=tf.ones(N), name='cnt')

# Recognition (q) model

In [9]:
with tf.variable_scope("qmodel"):
    q_A = ed.models.NormalWithSoftplusSigma(mu=tf.Variable(tf.random_normal((NU,))), 
                                            sigma=tf.Variable(tf.random_uniform((NU,))),
                                            name='A')
    q_B = ed.models.NormalWithSoftplusSigma(mu=tf.Variable(tf.random_normal((NU, P))), 
                                            sigma=tf.Variable(tf.random_uniform((NU, P))),
                                            name='B')
    q_C = ed.models.NormalWithSoftplusSigma(mu=tf.Variable(tf.random_normal((NU, K))), 
                                            sigma=tf.Variable(tf.random_uniform((NU, K))),
                                            name='C')
    q_Z = ed.models.BernoulliWithSigmoidP(p=tf.Variable(tf.random_normal((NS, K))), name='Z')
    tf.scalar_summary('mean_q_Z', tf.reduce_mean(tf.to_float(Z)))

    q_delta = ed.models.BetaWithSoftplusAB(a=tf.Variable(1 + tf.random_uniform((K,))),
                                           b=tf.Variable(1 + tf.random_uniform((K,))),
                                           name='delta')
    tf.scalar_summary('mean_q_delta', tf.reduce_mean(q_delta))

    q_lam = ed.models.NormalWithSoftplusSigma(mu=tf.Variable(tf.random_normal((N,))),
                                              sigma=tf.Variable(tf.random_uniform((N,))),
                                              name='lam')
    tf.scalar_summary('mean_q_lam', tf.reduce_mean(q_lam))

    q_sig = ed.models.NormalWithSoftplusSigma(mu=tf.Variable(-0.1 * tf.random_uniform((1,))),
                                              sigma=tf.Variable(tf.random_uniform((1,))),
                                              name='sig')
    tf.scalar_summary('mean_q_sig', tf.reduce_mean(q_sig))


# Do variational inference

In [10]:
data = {cnt: count}
inference = ed.KLqp({A: q_A, B: q_B, C: q_C, Z: q_Z, sig: q_sig, delta: q_delta, lam: q_lam}, data)

In [11]:
init = tf.initialize_all_variables()

# Notes before inference:

- The `logdir` keyword specifies the place to put the log file (assuming you've instrumented the code to save events, etc.). If a subdirectory is given, pointing Tensorboard at the parent directory allows you to compare across subdirectories (runs).
    - I'm using the `jmp/instrumented` branch of the `jmxpearson/edward` fork
- I had to lower the learning rate in Adam to avoid NaNs early on in learning. Gradient clipping might solve the same problem.
- I'm currently using "all" the data, but this should probably be switched to minibatches.
- I've used `n_samples` = 1, 5, 10, and 25, which all seem pretty similar after 10k iterations. 

In [12]:
inference.run(n_iter=1000, n_print=100, n_samples=1, logdir='data/run1',
             optimizer=tf.train.AdamOptimizer(1e-4))

Iteration    1 [  0%]: Loss = 26108448.000
Iteration  100 [ 10%]: Loss = 6635718.500
Iteration  200 [ 20%]: Loss = 33685660.000
Iteration  300 [ 30%]: Loss = 5678695.000
Iteration  400 [ 40%]: Loss = 23927398.000
Iteration  500 [ 50%]: Loss = 5479865.000
Iteration  600 [ 60%]: Loss = 8164009.000
Iteration  700 [ 70%]: Loss = 4888656.500
Iteration  800 [ 80%]: Loss = 7993082.500
Iteration  900 [ 90%]: Loss = 16878524.000
Iteration 1000 [100%]: Loss = 25808708.000
